# Setup

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
main_path = '/mnt/raid/ni/agnessa/RSA/'



# Define the function to create filenames

In [ ]:
def getFileName(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"       

# Get the lower triangular

In [ ]:
def get_upper_triangular(rdm):
    num_conditions = rdm.shape[0] #num samples
    print(num_conditions)
    return rdm[np.triu_indices(num_conditions,1)] #take all above the main diagonal (excluding it), returns flattened version

# Load the Model RDMs

In [ ]:
def getAverageCorrelation(trained_on,tested_on,layer_idx_rows,layer_idx_columns):
    model_rdm_filename = os.path.join(main_path,trained_on,tested_on,'Model_RDM',getFileName(10000,'Model_RDM'))
    model_rdm = np.load(model_rdm_filename)
    upper_triangular = get_upper_triangular(model_rdm[layer_idx_rows,layer_idx_columns])
    avg_correlation = np.mean(upper_triangular)
    return avg_correlation

In [ ]:
model_name = 'resnet50'
layer_name = 'all'
avg = getAverageCorrelation('Objects','',range(0,9),range(0,9))
# trained_objects_path = os.path.join(main_path,'Objects')
# trained_scenes_path = os.path.join(main_path,'Scenes')

#error in upper triangular: too many indices